In [1]:
import sys
from influx import Influx
sys.path.append("/home/jupyter-admin/bdpoc-datahub/visualize")
from visualize.configs.constants import VIBRATION_THRESHOLDS, VIBRATION_DURATIONS

In [7]:
fileHandle = open("./vibration-sample.csv")
csvData = fileHandle.read()

query = f'''
import "csv"
data = csv.from(csv: "{csvData}")
alarm = data |> stateCount(fn: (r) => r._value > {VIBRATION_THRESHOLDS["ALARM"]} and r._value <= {VIBRATION_THRESHOLDS["SHUTDOWN"]})
    |> filter(fn: (r) => r.stateCount > {VIBRATION_DURATIONS["ALARM"]})
    |> set(key: "alarmType", value: "ALARM")
shutdown = data |> stateCount(fn: (r) => r._value > {VIBRATION_THRESHOLDS["SHUTDOWN"]})
    |> filter(fn: (r) => r.stateCount > {VIBRATION_DURATIONS["SHUTDOWN"]})
    |> set(key: "alarmType", value: "CRITICAL")
union(tables: [alarm, shutdown])
'''

Influx().setRawQuery(query).asDataFrame()

,result,table,_start,_stop,_time,_value,_field,_measurement,stateCount,alarmType
0,_result,0,2023-09-24 22:52:50+07:00,2023-09-24 22:52:50+07:00,2023-09-24 22:52:50+07:00,4.05,HT_VI_4101B.PV,phdpeer,2,ALARM


In [6]:
fileHandle.close()

In [4]:
VIBRATION_DURATIONS["ALARM"]

1